# Notebook 2: Expériences sur les modèles

**Auteurs:**  

Akram Farihi, Sami Abloui, Amalya Mourih 

**Objectifs de notebook:**

Dans ce notebook, nous réalisons l'entraînement sur les modèles choisis avec accomagnement de MLFlows et le reste. Nous avons le code preprocessing.py qui permettra de faire le prétraiment des données. et nous allons effectue nis exp



In [ ]:
# les lib importé 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline


Preprocessing data avant de pouvoir l'entraineer

In [ ]:

X = df.drop(columns=['Sleep_Quality'])
y = df['Sleep_Quality']

# appliquer une répartition stratifiée car nos données sont déséquilibrées

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

Nous allons ensuite construire un pipeline afin d’automatiser la normalisation des données, l’entraînement des modèles et la définition des bonnes grilles de paramètres. Nous avons choisi d’utiliser des modèles de classification plutôt que des modèles de régression, car notre objectif est de prédire une classe. Trois approches seront donc évaluées :

- SVM, un classifieur performant et adapté aux jeux de données de taille moyenne.

- RandomForest, un modèle simple, non paramétrique et efficace, qui classe les instances selon la proximité de leurs voisins.

- MLP, un réseau de neurones adapté aux données tabulaires, contrairement aux CNN destinés aux images et aux RNN conçus pour les données séquentielles.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

pipelines = {
    "": Pipeline([
        ("scaler", MinMaxScaler),
        ("model", RandomForestClassifier())
    ]),
    "svm": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVC())
    ]),
    "mlp": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPClassifier(max_iter=300))
    ])
}

param_grids = {
    "randomforest": {
        "model__n_estimators": [100, 200, 300],      
        "model__max_depth": [None, 5, 10, 20],       
        "model__min_samples_split": [2, 5, 10],      
        "model__min_samples_leaf": [1, 2, 4],       
        "model__bootstrap": [True, False]            
    },
    "svm": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["rbf", "linear"]
    },
    "mlp": {
        "model__hidden_layer_sizes": [(50,), (100,), (50,50)],
        "model__activation": ["relu", "tanh"],
        "model__alpha": [0.0001, 0.001]
    }
}



In [ ]:
from sklearn.model_selection import GridSearchCV

best_models = {}
best_scores = {}

for name in pipelines:
    print("Recherche modèle :", name)

    grid = GridSearchCV(
        estimator=pipelines[name],
        param_grid=param_grids[name],
        cv=3,
        scoring="accuracy",
        n_jobs=-1
    )
    
    grid.fit(X_train, y_train)
    
    best_models[name] = grid.best_estimator_
    best_scores[name] = grid.best_score_
    
    print("Meilleurs paramètres :", grid.best_params_)
    print("Score CV :", grid.best_score_)
    print()


In [ ]:
best_model_name = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_name]

print("Meilleur modèle :", best_model_name)
